Templates
---------

In [ ]:
# Template portfolio data: past operations/transactions. Replace with actual data.

Ops =      {# Original portfolio transactions
            1: ['SPY', 100.0, 10.00, '', 'BUY', '2018-03-30', 'USD', '', '', 1.0] 
           }

# Template portfolio data: allocation targets
Pf =        { 'SPY': [100.0, 0.0, 0.0, 0.0] 
            }

In [ ]:
# Template usernames and API keys. Replace with actual username and API keys.
tiingo_api_key = "1234"
plotly_api_key = "1234"
plotly_username = 'johndoe'

Calculations
-----------

In [ ]:
# Import non-charting libraries
import pandas as pd
import numpy as np
from datetime import datetime
import pandas_datareader as pdr
from pandas_datareader.tiingo import TiingoDailyReader # pandas_datareader for Tiingo data feed
import tulipy as ty # Tulipy for calculating technical indicators like Williams %R
from copy import deepcopy

In [ ]:
# Constants
cols = ['open', 'high', 'low', 'close']

Buy = 'BUY'
Sell = 'SELL'

OpTicker = 0
OpQuantity = 1
OpCost = 2
OpBroker = 3
OpType = 4
OpDate = 5
OpCurrency = 6
OpRealTicker = 7
OpRealCurrency = 8
OpRealFactor = 9

PfTarget = 0
PfQuantity = 1
PfCostBasis = 2
PfCost = 3

In [ ]:
# Update portfolio with all past operations
for op in Ops.keys():
    if Ops[op][OpType] == Buy:
        Pf[Ops[op][OpTicker]][PfQuantity] = Pf[Ops[op][OpTicker]][PfQuantity] + Ops[op][OpQuantity]
        Pf[Ops[op][OpTicker]][PfCostBasis] = Pf[Ops[op][OpTicker]][PfCostBasis] + Ops[op][OpQuantity]*Ops[op][OpCost]
    elif Ops[op][OpType] == Sell:
        Pf[Ops[op][OpTicker]][PfQuantity] = Pf[Ops[op][OpTicker]][PfQuantity] - Ops[op][OpQuantity]
        Pf[Ops[op][OpTicker]][PfCostBasis] = Pf[Ops[op][OpTicker]][PfCostBasis] - Ops[op][OpQuantity]*Ops[op][OpCost]        
for tick in Pf.keys():
    if Pf[tick][PfQuantity] > 0.0:
        Pf[tick][PfCost] = Pf[tick][PfCostBasis]/Pf[tick][PfQuantity]
    else:
        Pf[tick][PfCost] = 0.0        
#Pf

In [ ]:
timespan = 1 # Last 1 year back from today is the default timespan
end = datetime.today()
start = datetime(end.year - timespan, end.month, end.day)

# Read ticker data from Tiingo and create OHLC and close series
ohlc = []
close = []
for tick in Pf.keys():
    data = TiingoDailyReader(tick, start, end, api_key=tiingo_api_key).read()
    ohlc.append(data.loc[tick][cols])
    close.append(data.loc[tick]['close'])
    
# Create OHLC and Close DataFrames
portfolioOHLC = pd.concat(ohlc, axis=1, keys=Pf.keys())
portfolioClose = pd.concat(close, axis=1, keys=Pf.keys())

#portfolioOHLC.tail(5)
#portfolioClose.head(5)

In [ ]:
# Williams %R period set to 14 days
willrPeriod = 14
portfolioWillr = {}

for tick in Pf.keys():
    willr14 = ty.willr(portfolioOHLC[tick]['high'].values, 
                       portfolioOHLC[tick]['low'].values, 
                       portfolioOHLC[tick]['close'].values, 
                       willrPeriod)
    portfolioWillr[tick] = np.pad(willr14, (willrPeriod-1,0), 'constant', constant_values=np.nan)

#print(portfolioWillr)

In [ ]:
#portfolioClose[list(Pf.keys())[0]].describe()

In [ ]:
# Compute the main portfolio metrics

metricNames = ['MaxReturnPct', 'MinReturnPct', 'Max', 'Mean', 
               'Min', 'Price', 'PriceChange', 'MarketValue', 'Cost Basis',
               'P&L', 'Percentage', 'DayChange', 'WillR'
              ]
metrics = []
for tick in Pf.keys():
    returns = portfolioClose[tick].pct_change()*100.0
    metrics.append(pd.DataFrame({tick: 
                                 [returns.max(),
                                  returns.min(),
                                  portfolioClose[tick].max(axis=0), 
                                  portfolioClose[tick].mean(axis=0), 
                                  portfolioClose[tick].min(axis=0), 
                                  portfolioClose[tick][-1],
                                  portfolioClose[tick][-1]-portfolioClose[tick][-2],
                                  portfolioClose[tick][-1]*Pf[tick][PfQuantity],
                                  Pf[tick][PfCostBasis],
                                  (portfolioClose[tick][-1]*Pf[tick][PfQuantity])-Pf[tick][PfCostBasis],
                                  0,
                                  (portfolioClose[tick][-1]-portfolioClose[tick][-2])*Pf[tick][PfQuantity],
                                  portfolioWillr[tick][-1]
                                 ]},  
                                index=metricNames))

portfolioMetrics = pd.concat(metrics, axis=1)
#portfolioMetrics

In [ ]:
# Compute portfolio market value and actual percentage for each ticker

portfolioValue = 0
percentages = []
for tick in Pf.keys():
    portfolioValue = portfolioValue + portfolioMetrics.loc['MarketValue'][tick]
for tick in Pf.keys():
    portfolioMetrics.loc['Percentage'][tick] = (portfolioMetrics.loc['MarketValue'][tick]/portfolioValue*100)     
   
portfolioMetrics

In [ ]:
# Actual vs. target percentages for each ticker in the portfolio
pct = []
for tick in Pf.keys():
    pct.append([portfolioMetrics.loc['Percentage'][tick], Pf[tick][PfTarget]])

alloc = pd.DataFrame(pct, index=Pf.keys(), columns=['Pct Actual', 'Pct Target'])
alloc

In [ ]:
# Oversold tickers as candidates for rebalancing, i.e. purchasing the quantities with *negative* totals
oversold = -50 # Williams %R oversold
candidates = []
keys = []
for tick in Pf.keys():
        if portfolioMetrics.loc['WillR'][tick] <= oversold:
            candidates.append([portfolioMetrics.loc['WillR'][tick], 
                               alloc.loc[tick]['Pct Actual'],
                               alloc.loc[tick]['Pct Target'],
                               alloc.loc[tick]['Pct Actual']-alloc.loc[tick]['Pct Target'],
                               portfolioValue*(alloc.loc[tick]['Pct Actual']-alloc.loc[tick]['Pct Target'])/100,
                               round((portfolioValue*(alloc.loc[tick]['Pct Actual']-alloc.loc[tick]['Pct Target'])/100)/portfolioClose[tick][-1]),
                               portfolioClose[tick][-1],
                               portfolioMetrics.loc['P&L'][tick],
                               ((portfolioClose[tick][-1] / portfolioMetrics.loc['Min'][tick])-1)*100,
                               (1-(portfolioClose[tick][-1] / portfolioMetrics.loc['Max'][tick]))*100
                             ])
            keys.append(tick)

rebalanceBuy = pd.DataFrame(candidates, index=keys, columns=['WillR', 'Pct Actual', 'Pct Target', 
                                                             'Pct Diff', 'Money Diff', 'Quantity', 
                                                             'Price', 'P&L', 'Pct > Min', 'Pct < Max'
                                                            ])
rebalanceBuy = rebalanceBuy.sort_values(by=['Quantity'])
rebalanceBuy

In [ ]:
# Overbought tickers as candidates for rebalancing, i.e. selling the quantities with negative totals
candidates = []
keys = []
for tick in Pf.keys():
        if portfolioMetrics.loc['WillR'][tick] > oversold:
            candidates.append([portfolioMetrics.loc['WillR'][tick], 
                               alloc.loc[tick]['Pct Actual'],
                               alloc.loc[tick]['Pct Target'],
                               alloc.loc[tick]['Pct Actual']-alloc.loc[tick]['Pct Target'],
                               portfolioValue*(alloc.loc[tick]['Pct Actual']-alloc.loc[tick]['Pct Target'])/100,
                               round((portfolioValue*(alloc.loc[tick]['Pct Actual']-alloc.loc[tick]['Pct Target'])/100)/portfolioClose[tick][-1]),
                               portfolioClose[tick][-1],
                               portfolioMetrics.loc['P&L'][tick],
                               ((portfolioClose[tick][-1] / portfolioMetrics.loc['Min'][tick])-1)*100,
                               (1-(portfolioClose[tick][-1] / portfolioMetrics.loc['Max'][tick]))*100
                             ])
            keys.append(tick)

rebalanceSell = pd.DataFrame(candidates, index=keys, columns=['WillR', 'Pct Actual', 'Pct Target', 
                                                              'Pct Diff', 'Money Diff', 'Quantity', 
                                                              'Price', 'P&L', 'Pct > Min', 'Pct < Max'
                                                             ])
rebalanceSell = rebalanceSell.sort_values(by=['Quantity'], ascending=False)
rebalanceSell

In [ ]:
# Template, replace with your actual operations/transactions
newOps =    {# New portfolio transactions /!\ APPLY REAL FACTOR TO CONVERT FROM US TO FOREIGN QUANTITIES /!\
            2: ['SPY', 10.0, portfolioClose['SPY'][-1], '', 'SELL', '2018-04-02', 'USD', '', '', 1.0]
            }

In [ ]:
# Portfolio sensitivity analysis (assuming no brand new portfolio ticker is bought)

newPf = deepcopy(Pf)

# Update portfolio with all past operations
for op in newOps.keys():
    if newOps[op][OpType] == Buy:
        newPf[newOps[op][OpTicker]][PfQuantity] = newPf[newOps[op][OpTicker]][PfQuantity] + newOps[op][OpQuantity]
        newPf[newOps[op][OpTicker]][PfCostBasis] = newPf[newOps[op][OpTicker]][PfCostBasis] + newOps[op][OpQuantity]*newOps[op][OpCost]
    elif newOps[op][OpType] == Sell:
        newPf[newOps[op][OpTicker]][PfQuantity] = newPf[newOps[op][OpTicker]][PfQuantity] - newOps[op][OpQuantity]
        newPf[newOps[op][OpTicker]][PfCostBasis] = newPf[newOps[op][OpTicker]][PfCostBasis] - newOps[op][OpQuantity]*newOps[op][OpCost]    
for tick in newPf.keys():
    if newPf[tick][PfQuantity] > 0.0:
        newPf[tick][PfCost] = newPf[tick][PfCostBasis] / newPf[tick][PfQuantity]
    else:
        newPf[tick][PfCost] = 0.0
    newPf[tick].append(portfolioClose[tick][-1] * newPf[tick][PfQuantity])
    newPf[tick].append(0.0) # Pct Actual
    newPf[tick].append(0.0) # Pct Diff
    

# Format portfolio as DataFrame for better readability
newAlloc = pd.DataFrame.from_dict(newPf, orient='index')
newAlloc.columns = ['Pct Target', 'Quantity', 'Cost Basis', 'Cost', 'Market Value', 'Pct Actual', 'Pct Diff']

# Recalculate portfolio percentages
newPfValue = newAlloc['Market Value'].sum()
for tick in newPf.keys():
    newAlloc.loc[tick]['Pct Actual'] = newAlloc.loc[tick]['Market Value'] / newPfValue * 100;
    newAlloc.loc[tick]['Pct Diff'] = newAlloc.loc[tick]['Pct Actual'] - newAlloc.loc[tick]['Pct Target'];

# Sort by percentage differences
newAlloc = newAlloc.sort_values(by=['Pct Diff'])
newAlloc
#newPfValue

In [ ]:
# Calculate returns (daily percent change) for each ticker
returns = portfolioClose.pct_change()*100
#returns.head(5)

In [ ]:
# Calculate Williams %R table with all tickers
qfTicker = list(Pf.keys())[0]
willrClose = pd.DataFrame(portfolioClose[qfTicker]) # FIXME: ugly hack to insert first column (ticker price) with date index

i = 0
for tick in Pf.keys():
    willrClose.insert(i, tick + " WILLR", portfolioWillr[tick], allow_duplicates=False)
    i += 1
willrClose = willrClose.drop(columns=[qfTicker], axis=1) # FIXME: ugly hack to get rid of first column (ticker price)
#willrClose.tail(10)

In [ ]:
# Calculate aggregate portfolio historical performance
hist = []
for tick in Pf.keys():
    hist.append(portfolioClose[tick]*Pf[tick][PfQuantity])
    
histClose = pd.DataFrame(hist).sum(axis=0)
histClose.tail(1)

In [ ]:
# Read benchmark ticker data from Tiingo and create close series
benchmark = 'SPY'
close = []
data = TiingoDailyReader(benchmark, start, end, api_key=tiingo_api_key).read()
close.append(data.loc[benchmark]['close'])
benchmarkClose = pd.concat(close, axis=1, keys=[benchmark])
#benchmarkClose.describe()

In [ ]:
# Uncomment to work only on data and avoid over-using Plot.ly 
# thisoperationwillfail()

Charts
------

In [ ]:
# Import charting libraries
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.plotly as py
from plotly import tools
import cufflinks as cf

# Set up Plot.ly - requires free account with username and API key
plotly.tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key)
cf.set_config_file(world_readable=True)

In [ ]:
# Grouped bar chart of ticker market value percentage of portfolio, actual vs. target
alloc.iplot(kind='bar', title="Portfolio Percentages", filename='PortfolioPercentages')

In [ ]:
# All closing prices on the same chart with log y axis
layout = go.Layout(
    autosize=True,
    yaxis=dict(type='log', title='Close', showticklabels=False),
)

data = []
for tick in Pf.keys():
    data.append(go.Scatter(x = portfolioClose.index, y = portfolioClose[tick].values, name=tick))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PortfolioClosingPrice', title="Portfolio Closing Price")

In [ ]:
# Multi-raph closing prices for portfolio tickers
# FIXME: log y axis for subplots - how?
layout = go.Layout(autosize=True,
    yaxis=dict(
        autorange=True
    ),
)
portfolioClose.iplot(layout=layout, filename='PortfolioClosingPrice', theme='pearl', shape=(1, len(Pf)), subplots = True, subplot_titles=True, shared_xaxes=True, fill=True, title="Portfolio Closing Price")

In [ ]:
# Technical analysis charts for one specific ticker - replace with actual ticker name, e.g. 'SPY'
qfTicker = 'SPY'

qf=cf.QuantFig(portfolioOHLC[qfTicker][cols],title='Technical Analysis',legend='top',name=qfTicker)
qf.add_ema([50,200],width=2,color=['green','lightgreen'],legendgroup=True)
qf.add_rsi(periods=14,color='java')
qf.add_bollinger_bands(periods=21,boll_std=2,colors=['magenta','grey'],fill=True)
#qf.add_volume() # Volume not stored in table
qf.add_macd()
qf.iplot(rangeslider=True, up_color='green',down_color='red',filename='PortfolioTechnicalAnalysis')

In [ ]:
# Distribution of returns box chart
returns.iplot(kind='box', title = "Portfolio Returns Distribution", filename='PortfolioReturnsDistribution')

In [ ]:
# Distribution of returns curve and rug (same information as the box chart)
py.iplot(ff.create_distplot([returns[c].dropna() for c in returns.columns], returns.columns, bin_size=.05, show_hist=False), filename='PortfolioReturnsDistributionCurve')

In [ ]:
# Heatmap of correlations between portfolio tickers based on returns
returns.corr().iplot(kind = 'heatmap', title = "Portfolio Correlations", colorscale = 'YlGnBu', filename='PortfolioReturnsCorrelations')

In [ ]:
# Chart portfolio components relative returns
relativeReturns = portfolioClose.apply(lambda x: ((x / x[0])-1)*100)
relativeReturns.iplot(rangeslider=True, title = "Portfolio Relative Returns", filename='PortfolioRelativeReturns')

In [ ]:
# Dashboard of Williams %R graphs for all tickers
layout = go.Layout(autosize=True,
    width=1000,
    height=1000,
)
willrClose.iplot(layout=layout, subplots=True, shape=(len(Pf)+1,1), vertical_spacing=.02,fill=True, title = "Portfolio Williams %R", filename='WillRClose')

In [ ]:
# Chart of portfolio historical performance with log axis
layout = go.Layout(autosize=True,
    yaxis=dict(
        type='log',
        autorange=True
    ),
)
histClose.iplot(layout=layout, filename='HistoryClosingPrice', theme='pearl', fill=True, title="Historical Portfolio Closing Price")

In [ ]:
# Chart portfolio relative returns
benchmarkReturns = benchmarkClose.apply(lambda x: ((x / x[0])-1)*100)
portfolioAggClose = pd.DataFrame(histClose, columns=['Close'])
portfolioAggReturns = portfolioAggClose.apply(lambda x: ((x / x[0])-1)*100)
benchmarkReturns = pd.concat([benchmarkReturns, portfolioAggReturns], axis=1, keys=['Benchmark', 'Portfolio'])
benchmarkReturns.iplot(rangeslider=True, title = "Portfolio Benchmark Returns", filename='PortfolioBenchmarkReturns')